# import 初始值設定 augmentations

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os, time, torchprofile
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import torch.optim as optim


num_classes = 50
epochs = 30
batch_size = 32
# test_batch_size = 32
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
# Define a custom dataset class
class ImageNetDataset(Dataset):
    def __init__(self, data_file, root_dir, transform=None):
        """
        Args:
            data_file (string): Path to the file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        # Read the data file
        with open(data_file, 'r') as file:
            self.data = file.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Process each line
        line = self.data[idx].strip()
        img_path, label = line.split()
        # Complete image path
        img_path = os.path.join(self.root_dir, img_path)
        image = Image.open(img_path).convert('RGB')
        label = int(label)
        
        if self.transform:
            image = self.transform(image)

        return image, label
    
# Define transformations

#基礎影像處理
basic_augmentations = transforms.Compose([
    transforms.Resize(84),
    transforms.CenterCrop(84),
    transforms.ToTensor()
])

train_augmentations = transforms.Compose([
    transforms.RandomResizedCrop(84),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

val_augmentations = transforms.Compose([
    transforms.Resize(84),
    transforms.CenterCrop(84),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_augmentations = transforms.Compose([
    transforms.Resize(84),
    transforms.CenterCrop(84),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


# 把 train validation data load 進來

In [3]:
# Create dataset instances
train_dataset = ImageNetDataset(data_file='data/images/train.txt', root_dir='data/images', transform=train_augmentations)
val_dataset = ImageNetDataset(data_file='data/images/val.txt', root_dir='data/images', transform=val_augmentations)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# 我的設計 3-layer ResAttentionNet model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EnhancedResidualAttentionModel(nn.Module):
    def __init__(self, num_classes=50):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.residual_block1 = self._make_layer(64, 256)
        self.attention_module1 = AttentionModule_stage1(256)
        self.residual_block2 = self._make_layer(256, 512)

        self.mpool2 = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.mpool1(x)
        x = self.residual_block1(x)
        x = self.attention_module1(x)
        x = self.residual_block2(x)
        x = self.mpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels // 4, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels // 4)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels // 4, out_channels // 4, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels // 4)

        self.conv3 = nn.Conv2d(out_channels // 4, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)
        return out

class AttentionModule_stage1(nn.Module):
    def __init__(self, channels):
        super(AttentionModule_stage1, self).__init__()
        self.residual_blocks = ResidualBlock(channels, channels)
        self.trunk_branches = nn.Sequential(
            ResidualBlock(channels, channels),
            ResidualBlock(channels, channels)
        )
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        residual = self.residual_blocks(x)
        out_trunk = self.trunk_branches(x)

        # Softmax for simplicity
        attention_weights = self.softmax(out_trunk)
        out = torch.mul(attention_weights, residual)
        out = out + out_trunk
        return out




In [5]:
model = EnhancedResidualAttentionModel(num_classes=50)
model.cuda()

EnhancedResidualAttentionModel(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (mpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (residual_block1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
   

# 開始訓練

In [6]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))


def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return correct / len(val_loader.dataset)


best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model,'best2_3layer.pt')
        print('Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

print('Final Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [02:51<00:00,  5.76it/s, accuracy=11.7, loss=3.39] 


Train Epoch: 1, Loss: 3.426510, Accuracy: 11.74%


Validation Epoch #1: 100%|██████████| 8/8 [00:00<00:00,  8.27it/s, accuracy=14.7, loss=3.39] 


Validation Set: Average Loss: 3.3898, Accuracy: 14.67%
Best Accuracy: 14.67%


Training Epoch #2: 100%|██████████| 990/990 [02:51<00:00,  5.78it/s, accuracy=18.8, loss=2.76]


...

Validation Set: Average Loss: 1.2517, Accuracy: 64.44%


Training Epoch #30: 100%|██████████| 990/990 [02:50<00:00,  5.80it/s, accuracy=54.4, loss=1.53]


Train Epoch: 30, Loss: 1.602591, Accuracy: 54.38%


Validation Epoch #30: 100%|██████████| 8/8 [00:00<00:00,  8.24it/s, accuracy=63.8, loss=1.26] 

Validation Set: Average Loss: 1.2554, Accuracy: 63.78%
Final Best Accuracy: 64.89%


# Resnet34 模型

In [7]:
import torchvision.models as models

model = models.resnet34(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)  # CIFAR-10有10個類別
model.cuda()

/home/r26121073/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/r26121073/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 訓練 Resnet34 模型

In [9]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))


def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return correct / len(val_loader.dataset)


best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model,'resnet34_best.pt')
        print('Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

print('Final Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [02:57<00:00,  5.59it/s, accuracy=4.41, loss=3.87] 


Train Epoch: 1, Loss: 3.858497, Accuracy: 4.41%


Validation Epoch #1: 100%|██████████| 8/8 [00:00<00:00,  8.12it/s, accuracy=5.78, loss=3.75] 


Validation Set: Average Loss: 3.7463, Accuracy: 5.78%
Best Accuracy: 5.78%


Training Epoch #2: 100%|██████████| 990/990 [02:55<00:00,  5.66it/s, accuracy=8.41, loss=3.58] 


...

Validation Set: Average Loss: 1.3982, Accuracy: 55.33%


Training Epoch #30: 100%|██████████| 990/990 [03:01<00:00,  5.45it/s, accuracy=54.4, loss=1.2] 


Train Epoch: 30, Loss: 1.556348, Accuracy: 54.45%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  7.79it/s, accuracy=58.4, loss=1.36] 

Validation Set: Average Loss: 1.3621, Accuracy: 58.44%
Final Best Accuracy: 58.67%


# 把兩個分別最好的 model load 進來

In [10]:
my_model = torch.load("best2_3layer.pt")
my_model.cuda

resnet_model = torch.load("resnet34_best.pt")
resnet_model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 把 test data load 進來

In [11]:
test_augmentations = transforms.Compose([
    transforms.Resize(84),
    transforms.CenterCrop(84),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_dataset = ImageNetDataset(data_file='data/images/test.txt', root_dir='data/images', transform=test_augmentations)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 開始測試比較

In [12]:
def test(model):
    model.eval()
    correct = 0
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total=len(test_loader), desc="Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, target in test_loader_iter:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy=100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    
    # 計算 Precision, Recall 和 F1-score
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    
    # 計算 FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).cuda())
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

ran_acc, ran_precision, ran_recall, ran_f1, ran_flops, ran_elapsed_time = test(my_model)
print(f"Accuracy: {ran_acc:.2f}%, Precision: {ran_precision:.4f}, Recall: {ran_recall:.4f}, F1 Score: {ran_f1:.4f}, FLOPS: {ran_flops:d}, Elapsed Time: {ran_elapsed_time:.2f} seconds")
resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(resnet_model)
print(f"Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.4f}, Recall: {resnet_recall:.4f}, F1 Score: {resnet_f1:.4f}, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:01<00:00,  7.73it/s, accuracy=62]  


Accuracy: 62.00%, Precision: 0.6273, Recall: 0.6200, F1 Score: 0.6123, FLOPS: 309248256, Elapsed Time: 1.04 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  7.79it/s, accuracy=59.8]


Accuracy: 59.78%, Precision: 0.5999, Recall: 0.5978, F1 Score: 0.5898, FLOPS: 612876800, Elapsed Time: 1.03 seconds
